In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from dagster import execute_pipeline

In [4]:
from processing_pipeline import process_show_notes

In [5]:
result = execute_pipeline(process_show_notes)

2020-04-18 19:51:15 - dagster - DEBUG - process_show_notes - 3dd04f7b-2e54-4f5d-ae8d-ac150db9d472 - PIPELINE_START - Started execution of pipeline "process_show_notes".
2020-04-18 19:51:15 - dagster - DEBUG - process_show_notes - 3dd04f7b-2e54-4f5d-ae8d-ac150db9d472 - ENGINE_EVENT - Executing steps in process (pid: 20233)
 event_specific_data = {"error": null, "marker_end": null, "marker_start": null, "metadata_entries": [["pid", null, ["20233"]], ["step_keys", null, ["['read_show_notes.compute', 'explode_people_mentioned.compute', 'extract_mentioned_peoples_names.compute', 'remove_missing_names.compute', 'clean_mentioned_peoples_full_names.compute', 'parse_mentioned_people_names.compute']"]]]}
2020-04-18 19:51:15 - dagster - DEBUG - process_show_notes - 3dd04f7b-2e54-4f5d-ae8d-ac150db9d472 - STEP_START - Started execution of step "read_show_notes.compute".
               solid = "read_show_notes"
    solid_definition = "read_show_notes"
            step_key = "read_show_notes.compute"

2020-04-18 19:51:18 - dagster - DEBUG - process_show_notes - 3dd04f7b-2e54-4f5d-ae8d-ac150db9d472 - STEP_SUCCESS - Finished execution of step "clean_mentioned_peoples_full_names.compute" in 34ms.
 event_specific_data = {"duration_ms": 34.692877000000344}
               solid = "clean_mentioned_peoples_full_names"
    solid_definition = "clean_mentioned_peoples_full_names"
            step_key = "clean_mentioned_peoples_full_names.compute"
2020-04-18 19:51:18 - dagster - DEBUG - process_show_notes - 3dd04f7b-2e54-4f5d-ae8d-ac150db9d472 - STEP_START - Started execution of step "parse_mentioned_people_names.compute".
               solid = "parse_mentioned_people_names"
    solid_definition = "parse_mentioned_people_names"
            step_key = "parse_mentioned_people_names.compute"
2020-04-18 19:51:18 - dagster - DEBUG - process_show_notes - 3dd04f7b-2e54-4f5d-ae8d-ac150db9d472 - STEP_INPUT - Got input "df" of type "Any". (Type check passed).
 event_specific_data = {"input_name": "df", 

In [6]:
dir(result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_construct_events_by_step_key',
 '_events_by_step_key',
 'event_list',
 'events_by_step_key',
 'output_for_solid',
 'pipeline',
 'reconstruct_context',
 'result_for_handle',
 'result_for_solid',
 'run_id',
 'solid_result_list',
 'step_event_list',
 'success']

In [7]:
df = result.result_for_solid('parse_mentioned_people_names').output_value()

In [8]:
df

,parsed_title,parsed_first,parsed_middle,parsed_last,parsed_suffix,parsed_nickname
0,,Margaret,Myfanwe,Joseph,,Vanne
0,,Mortimer,,Morris-Goodall,,
0,,Derek,,Bryceson,,
0,,Louis,,Leakey,,
0,,Mary,,Leakey,,
...,...,...,...,...,...,...
377,,Hazelden,Treatment,Center,,
379,,Henry,David,Thoreau,,
379,,Ayn,,Rand,,
379,,Anne,,Lamott,,


In [9]:
df = df.reset_index()
df = df.rename(columns={'index': 'origional_index'})

In [10]:
import recordlinkage

In [11]:
# Make record pairs, one for every combination of records
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(df)

In [12]:
print (len(df), len(candidate_links))

6876 23636250


In [13]:
compare_cl = recordlinkage.Compare()
compare_cl.string('parsed_first', 'parsed_first', threshold=0.85, label='first_name')
compare_cl.string('parsed_last', 'parsed_first', threshold=0.85, label='surname')

features = compare_cl.compute(candidate_links, df)

In [14]:
features.head(10)

first_name  surname
1 0         0.0      0.0
2 0         0.0      0.0
  1         0.0      0.0
3 0         0.0      0.0
  1         0.0      0.0
  2         0.0      0.0
4 0         0.0      0.0
  1         0.0      0.0
  2         0.0      0.0
  3         0.0      0.0

In [15]:
matches = features[features.sum(axis=1) > 1.9]
matches

first_name  surname
1247 880          1.0      1.0
4326 880          1.0      1.0
     1247         1.0      1.0

In [16]:
merged_results = matches.reset_index()
merged_results

,level_0,level_1,first_name,surname
0,1247,880,1.0,1.0
1,4326,880,1.0,1.0
2,4326,1247,1.0,1.0


In [17]:
df.head()

,origional_index,parsed_title,parsed_first,parsed_middle,parsed_last,parsed_suffix,parsed_nickname
0,0,,Margaret,Myfanwe,Joseph,,Vanne
1,0,,Mortimer,,Morris-Goodall,,
2,0,,Derek,,Bryceson,,
3,0,,Louis,,Leakey,,
4,0,,Mary,,Leakey,,


In [18]:
pc = df.reset_index()
pc = pc.rename(columns={'index': 'comparison_index'})
pc.head()

,comparison_index,origional_index,parsed_title,parsed_first,parsed_middle,parsed_last,parsed_suffix,parsed_nickname
0,0,0,,Margaret,Myfanwe,Joseph,,Vanne
1,1,0,,Mortimer,,Morris-Goodall,,
2,2,0,,Derek,,Bryceson,,
3,3,0,,Louis,,Leakey,,
4,4,0,,Mary,,Leakey,,


In [19]:
first_merge = merged_results.merge(pc, left_on='level_1', right_on='comparison_index')

In [20]:
first_merge

,level_0,level_1,first_name,surname,comparison_index,origional_index,parsed_title,parsed_first,parsed_middle,parsed_last,parsed_suffix,parsed_nickname
0,1247,880,1.0,1.0,880,37,,Galileo,,Galilei,,
1,4326,880,1.0,1.0,880,37,,Galileo,,Galilei,,
2,4326,1247,1.0,1.0,1247,53,,Galileo,,Galilei,,


In [21]:
second_merge = first_merge.merge(pc, left_on='level_0', right_on='comparison_index')

In [22]:
second_merge

,level_0,level_1,first_name,surname,comparison_index_x,origional_index_x,parsed_title_x,parsed_first_x,parsed_middle_x,parsed_last_x,parsed_suffix_x,parsed_nickname_x,comparison_index_y,origional_index_y,parsed_title_y,parsed_first_y,parsed_middle_y,parsed_last_y,parsed_suffix_y,parsed_nickname_y
0,1247,880,1.0,1.0,880,37,,Galileo,,Galilei,,,1247,53,,Galileo,,Galilei,,
1,4326,880,1.0,1.0,880,37,,Galileo,,Galilei,,,4326,173,,Galileo,,Galilei,,
2,4326,1247,1.0,1.0,1247,53,,Galileo,,Galilei,,,4326,173,,Galileo,,Galilei,,
